In [ ]:
import json
from datetime import datetime
from typing import Union, Dict, Any

try:
    import pandas as pd
except ImportError:
    pd = None

RAW = r'''{"status":"ok","data":{"aqi":43,"idx":5353,"attributions":[{"url":"https://www.deq.virginia.gov/","name":"Air Quality in Virginia","logo":"US-VirginiaDEQ.png"},{"url":"http://www.airnow.gov/","name":"Air Now - US EPA"},{"url":"https://waqi.info/","name":"World Air Quality Index Project"}],"city":{"geo":[38.48123,-77.3704],"name":"Widewater Elem. School - Widewater, Fredericksburg, USA","url":"https://aqicn.org/city/usa/virginia/widewater-elem.-school-widewater","location":""},"dominentpol":"pm25","iaqi":{"co":{"v":0.1},"dew":{"v":24},"h":{"v":66},"no2":{"v":1.2},"o3":{"v":17.1},"p":{"v":1013.5},"pm10":{"v":15},"pm25":{"v":43},"so2":{"v":0.3},"t":{"v":31},"w":{"v":3.6},"wg":{"v":14.1}},"time":{"s":"2025-08-17 14:00:00","tz":"-04:00","v":1755439200,"iso":"2025-08-17T14:00:00-04:00"},"forecast":{"daily":{"pm10":[{"avg":17,"day":"2025-08-15","max":17,"min":17},{"avg":24,"day":"2025-08-16","max":37,"min":18},{"avg":19,"day":"2025-08-17","max":24,"min":14},{"avg":14,"day":"2025-08-18","max":24,"min":7},{"avg":12,"day":"2025-08-19","max":13,"min":8},{"avg":16,"day":"2025-08-20","max":23,"min":12},{"avg":15,"day":"2025-08-21","max":23,"min":9},{"avg":20,"day":"2025-08-22","max":27,"min":17}],"pm25":[{"avg":62,"day":"2025-08-15","max":62,"min":61},{"avg":68,"day":"2025-08-16","max":85,"min":57},{"avg":60,"day":"2025-08-17","max":74,"min":47},{"avg":51,"day":"2025-08-18","max":75,"min":27},{"avg":31,"day":"2025-08-19","max":50,"min":22},{"avg":41,"day":"2025-08-20","max":60,"min":26},{"avg":43,"day":"2025-08-21","max":64,"min":24},{"avg":58,"day":"2025-08-22","max":73,"min":51}],"uvi":[{"avg":0,"day":"2025-08-16","max":0,"min":0},{"avg":1,"day":"2025-08-17","max":7,"min":0},{"avg":1,"day":"2025-08-18","max":4,"min":0},{"avg":1,"day":"2025-08-19","max":5,"min":0},{"avg":1,"day":"2025-08-20","max":5,"min":0},{"avg":1,"day":"2025-08-21","max":5,"min":0}]}},"debug":{"sync":"2025-08-18T04:23:11+09:00"}}}'''

def load_payload(payload: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
    """Accept raw JSON string or dict and return parsed dict."""
    if isinstance(payload, str):
        return json.loads(payload)
    return payload

def aqi_category(aqi: int) -> str:
    """US EPA-style AQI categories."""
    if aqi <= 50: return "Good (0–50)"
    if aqi <= 100: return "Moderate (51–100)"
    if aqi <= 150: return "Unhealthy for Sensitive Groups (101–150)"
    if aqi <= 200: return "Unhealthy (151–200)"
    if aqi <= 300: return "Very Unhealthy (201–300)"
    return "Hazardous (301–500)"

def summarize(payload: Union[str, Dict[str, Any]], use_pandas: bool = True):
    d = load_payload(payload)
    data = d.get("data", {})

    # Top-level info
    city = data.get("city", {})
    name = city.get("name", "Unknown location")
    lat, lon = (city.get("geo") or [None, None])[:2]
    aqi = data.get("aqi")
    aqi_cat = aqi_category(aqi) if aqi is not None else "N/A"
    dom = data.get("dominentpol", "N/A")
    time_iso = (data.get("time", {}) or {}).get("iso")
    local_time = time_iso
    try:
        # normalize for better display
        local_time = datetime.fromisoformat(time_iso).strftime("%Y-%m-%d %H:%M %Z") if time_iso else "N/A"
    except Exception:
        pass

    # IAQI small panel (key → value)
    iaqi = {
        k: v.get("v") for k, v in (data.get("iaqi", {}) or {}).items()
        if isinstance(v, dict) and "v" in v
    }

    # Attributions (names and URLs)
    attributions = [(a.get("name"), a.get("url")) for a in data.get("attributions", [])]

    # Forecast merge: pm25, pm10, uvi → by day
    daily = (data.get("forecast", {}) or {}).get("daily", {})
    def to_map(list_of_dicts, prefix):
        out = {}
        for row in list_of_dicts or []:
            day = row.get("day")
            if not day:
                continue
            out.setdefault(day, {})
            for stat in ("avg","min","max"):
                if stat in row:
                    out[day][f"{prefix}_{stat}"] = row[stat]
        return out

    pm25_map = to_map(daily.get("pm25"), "pm25")
    pm10_map = to_map(daily.get("pm10"), "pm10")
    uvi_map  = to_map(daily.get("uvi"),  "uvi")

    # Merge by keys (days)
    all_days = sorted(set(pm25_map) | set(pm10_map) | set(uvi_map))
    rows = []
    for day in all_days:
        row = {"day": day}
        row.update(pm25_map.get(day, {}))
        row.update(pm10_map.get(day, {}))
        row.update(uvi_map.get(day, {}))
        rows.append(row)

    # ----------- PRINT SUMMARY -----------
    print("=== Air Quality Summary ===")
    print(f"Location:     {name}")
    print(f"Coordinates:  {lat}, {lon}")
    print(f"Local Time:   {time_iso}")
    print(f"AQI:          {aqi}  |  Category: {aqi_cat}")
    print(f"Dominant Pol: {dom}")
    if iaqi:
        print("\nIAQI Panel (instantaneous indices):")
        # Order common keys first, then the rest
        common_order = ["pm25","pm10","o3","no2","so2","co","t","dew","h","p","w","wg"]
        ordered = sorted(iaqi.items(), key=lambda kv: (common_order.index(kv[0]) if kv[0] in common_order else 999, kv[0]))
        for k, v in ordered:
            print(f"  - {k}: {v}")

    if attributions:
        print("\nAttributions:")
        for name_, url_ in attributions:
            if url_:
                print(f"  • {name_} — {url_}")
            else:
                print(f"  • {name_}")

    # ----------- OPTIONAL: PANDAS TABLE -----------
    if pd is not None and use_pandas:
        df = pd.DataFrame(rows).sort_values("day")
        # Reorder columns for readability if present
        col_order = ["day",
                     "pm25_min","pm25_avg","pm25_max",
                     "pm10_min","pm10_avg","pm10_max",
                     "uvi_min","uvi_avg","uvi_max"]
        cols = [c for c in col_order if c in df.columns] + [c for c in df.columns if c not in col_order]
        df = df[cols]
        print("\n=== Forecast (Daily) ===")
        print(df.to_string(index=False))
        return df  # return DF for downstream use
    else:
        # Fallback plain text forecast
        if rows:
            print("\n=== Forecast (Daily) ===")
            for r in rows:
                print(
                    f"{r['day']}: "
                    f"PM2.5 avg/min/max={r.get('pm25_avg','-')}/{r.get('pm25_min','-')}/{r.get('pm25_max','-')}, "
                    f"PM10 avg/min/max={r.get('pm10_avg','-')}/{r.get('pm10_min','-')}/{r.get('pm10_max','-')}, "
                    f"UVI avg/min/max={r.get('uvi_avg','-')}/{r.get('uvi_min','-')}/{r.get('uvi_max','-')}"
                )
        return rows  # return list-of-dicts if pandas not available

# ---- Run on your payload ----
if __name__ == "__main__":
    summarize(RAW, use_pandas=True)


=== Air Quality Summary ===
Location:     Widewater Elem. School - Widewater, Fredericksburg, USA
Coordinates:  38.48123, -77.3704
Local Time:   2025-08-17T14:00:00-04:00
AQI:          43  |  Category: Good (0–50)
Dominant Pol: pm25

IAQI Panel (instantaneous indices):
  - pm25: 43
  - pm10: 15
  - o3: 17.1
  - no2: 1.2
  - so2: 0.3
  - co: 0.1
  - t: 31
  - dew: 24
  - h: 66
  - p: 1013.5
  - w: 3.6
  - wg: 14.1

Attributions:
  • Air Quality in Virginia — https://www.deq.virginia.gov/
  • Air Now - US EPA — http://www.airnow.gov/
  • World Air Quality Index Project — https://waqi.info/

=== Forecast (Daily) ===
       day  pm25_min  pm25_avg  pm25_max  pm10_min  pm10_avg  pm10_max  uvi_min  uvi_avg  uvi_max
2025-08-15        61        62        62        17        17        17      NaN      NaN      NaN
2025-08-16        57        68        85        18        24        37      0.0      0.0      0.0
2025-08-17        47        60        74        14        19        24      0.0      